# Refernece using state
## input h5ad 

In [6]:
import h5py
import os
os.environ['MPLBACKEND'] = 'Agg'

In [7]:
# Check the structure of our sample H5 files
with h5py.File('virtual_cell/predicted_only_10.h5ad', 'r') as f:
    print(type(f))
    print("Keys in root:", list(f.keys()))
    if 'obsm' in f:
        print("Keys in obsm:", list(f['obsm'].keys()))
        print("Keys in X:", list(f['X'].keys()))
    else:
        print("No 'obsm' group found")

<class 'h5py._hl.files.File'>
Keys in root: ['X', 'layers', 'obs', 'obsm', 'obsp', 'uns', 'var', 'varm', 'varp']
Keys in obsm: []


AttributeError: 'Dataset' object has no attribute 'keys'

In [3]:
! ls virtural_cell

predicted_only_10.h5ad


In [8]:

# Check the structure of one of COLAB H5 files
with h5py.File('/workspace/training_dataset/competition_support_set/competition_train.h5', 'r') as f:
    print(type(f))
    print("Keys in root:", list(f.keys()))
    if 'obsm' in f:
        print("Keys in obsm:", list(f['obsm'].keys()))
        print("Keys in X:", list(f['X'].keys()))
    else:
        print("No 'obsm' group found")

<class 'h5py._hl.files.File'>
Keys in root: ['X', 'layers', 'obs', 'obsm', 'obsp', 'uns', 'var', 'varm', 'varp']
Keys in obsm: []
Keys in X: ['data', 'indices', 'indptr']


## Model download
### 1. SE600M Download trained parameteres 
### 2. var_dims.pkl from ST-Tahoe [reference](https://github.com/ArcInstitute/state/issues/133)

In [43]:
import os
from huggingface_hub import snapshot_download

# Define the repository and local directory
repo_id = "arcinstitute/SE-600M"
local_dir = "virtual_cell/SE-600M/"

# Download all files from the repository
print(f"Downloading all files from {repo_id}...")
local_path = snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    local_dir_use_symlinks=False  # This ensures actual files are downloaded, not symlinks
)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

LICENSE.md: 0.00B [00:00, ?B/s]

protein_embeddings.pt:   0%|          | 0.00/411M [00:00<?, ?B/s]

MODEL_ACCEPTABLE_USE_POLICY.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

README.md:   0%|          | 0.00/823 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

se600m_epoch16.ckpt:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

se600m_epoch4.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

se600m_epoch4.ckpt:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

In [46]:
# SE600M doesn't contain var_dims.pkl but ST-Tahoe does and should be the model used for inference

repo_id = "arcinstitute/ST-Tahoe"
filename = "var_dims.pkl"

# Download the file, it will be cached and its local path returned
local_path = hf_hub_download(repo_id=repo_id, filename=filename)

print(f"File downloaded to: {local_path}")

var_dims.pkl:   0%|          | 0.00/206k [00:00<?, ?B/s]

File downloaded to: /root/.cache/huggingface/hub/models--arcinstitute--ST-Tahoe/snapshots/6fd3edc556f8587793035f907eb03a48867ae09f/var_dims.pkl


In [52]:
! ls virtual_cell/SE-600M/

LICENSE.md			protein_embeddings.pt
MODEL_ACCEPTABLE_USE_POLICY.md	se600m_epoch16.ckpt
MODEL_LICENSE.md		se600m_epoch4.ckpt
README.md			se600m_epoch4.safetensors
config.yaml			var_dims.pkl
model.safetensors


## Inferecne our sample
### Killed reason: 1.My MacPro (16G memory) might not enough memory for inference run. 

In [53]:
! state tx infer \
  --output "virtual_cell/prediction_SE600M_250618/prediction_only_10.h5ad" \
  --model_dir virtual_cell/SE-600M \
  --checkpoint virtual_cell/SE-600M/se600m_epoch16.ckpt \
  --adata "virtual_cell/predicted_only_10.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/SE-600M/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/SE-600M/se600m_epoch16.ckpt
Killed


In [54]:
# embedding first, but not solving the error.
! state emb transform \
  --model-folder virtual_cell/SE-600M \
  --input virtual_cell/predicted_only_10.h5ad \
  --output virtual_cell/predicted_only_10_emb.h5ad

INFO:state._cli._emb._transform:Using model checkpoint: virtual_cell/SE-600M/se600m_epoch4.ckpt
INFO:state._cli._emb._transform:Creating inference object
INFO:state._cli._emb._transform:Loading model from checkpoint: virtual_cell/SE-600M/se600m_epoch4.ckpt
Killed


## Wrong model ST-Parse logs for learning structure for ours and COLAB

In [39]:
! state tx infer \
  --output "virtual_cell/prediction_250617/prediction_only_10.h5ad" \
  --model_dir virtual_cell/ST_Parse \
  --checkpoint virtual_cell/ST_Parse/final.ckpt \
  --adata "virtual_cell/predicted_only_10.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/ST_Parse/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/ST_Parse/final.ckpt
PertSetsPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU

In [40]:
# using COLAB input for testing
! state tx infer \
  --output "virtual_cell/prediction_250617/prediction_only_10.h5ad" \
  --model_dir virtual_cell/ST_Parse \
  --checkpoint virtual_cell/ST_Parse/final.ckpt \
  --adata "training_dataset/competition_support_set/competition_val_template.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/ST_Parse/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/ST_Parse/final.ckpt
PertSetsPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU